# Policy Gradient Exercises

**NOTICE:**
1. You are allowed to work in groups of up to three people but **have to document** your group's\
 members in the top cell of your notebook.
2. **Comment your code**, explain what you do (refer to the slides). It will help you understand the topics\
 and help me understand your thinking progress. Quality of comments will be graded.
3. **Discuss** and analyze your results, **write-down your learnings**. These exercises are no programming\
 exercises it is about learning and getting a touch for these methods. Such questions might be asked in the\
 final exams.
 4. Feel free to **experiment** with these methods. Change parameters think about improvements, write down\
 what you learned. This is not only about collecting points for the final grade, it is about understanding\
  the methods.

In [1]:
# If you run on google-colab you have to install this package whenever you start a kernel
#
!pip install gymnasium
!pip install mujoco==2.3.0

KeyboardInterrupt: 

### Exercise 1 - REINFORCE

**Summary:** Implement the REINFORCE algorithm and use it to solve the ```CartPole-v1``` environment.


**Provided Code:** Feel free to re-use code from previous exercises.


**Your Tasks in this exercise:**
1. Implement REINFORCE
2. Solve the ```CartPole-v1``` environment.
    


### Exercise 2 - Deep Deterministic Policy Gradient (DDPG)

**Summary:** Implement the DDPG algorithm and use it to solve the ```Pusher-v4``` environment. If the   
physics do not work as supposed , you might have to explicitly install mujoco version 2.3.0.


**Provided Code:** Feel free to re-use code from previous exercises. Below I have provided you with   
an implementation for soft weight-updates using keras.


**Your Tasks in this exercise:**
1. Implement DDPG
2. Solve the ```Pusher-v4``` environment.
    

In [ ]:
def update_target_weights(source, target, tau=0.99):
    ''' Performs a soft update as:
        target <- tau * tar + (1-tau) * src
        This is the other way as in our previous implementation following the DDPG paper.
    '''
    for i in range(len(source.layers)):

        layer_weights_list_source = source.layers[i].get_weights()
        layer_weights_list_target = target.layers[i].get_weights()

        new_weights = []
        for (w_src, w_target) in zip(layer_weights_list_source, layer_weights_list_target):
            w_target = w_target* tau + (1.0-tau) * w_src
            new_weights.append(w_target)

        target.layers[i].set_weights(new_weights)